<a href="https://colab.research.google.com/github/Medonic21/GiPi_at_Home/blob/main/CodeT5_Training(GiPi_at_home).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install "transformers>=4.41.0,<5.0.0" "datasets>=2.0.0" "accelerate>=0.10.0"

In [3]:
!pip install sentence-transformers

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import load_dataset

In [6]:
model_name = "Salesforce/codet5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [7]:
dataset = load_dataset("code_search_net", "python")
# Replace "code_search_net" and "python" with your dataset and language

def preprocess_function(examples):
    inputs = [example for example in examples["func_code_string"]]
    targets = [example for example in examples["func_documentation_string"]]
    model_inputs = tokenizer(inputs, max_length=128, padding="max_length", truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

Map:   0%|          | 0/412178 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/22176 [00:00<?, ? examples/s]

Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [8]:
print(dataset['train'].column_names)

['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url']


In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    run_name="my-codet5-experiment",
    report_to="none",  # Disable wandb
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    # ... other training arguments ...
)

# Assuming you want to use 0.1% of the training dataset
smaller_train_dataset = tokenized_datasets["train"].select(range(int(len(tokenized_datasets["train"]) * 0.1)))

# Assuming you want to use 0.05% of the validation dataset
smaller_eval_dataset = tokenized_datasets["validation"].select(range(int(len(tokenized_datasets["validation"]) * 0.5)))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=smaller_train_dataset,
    eval_dataset=smaller_eval_dataset,
)

trainer.train()

Step,Training Loss
500,0.111300
1000,0.099100
1500,0.090800
2000,0.085700
2500,0.084300
3000,0.077100
3500,0.086500
4000,0.080700
4500,0.079400
5000,0.081700


TrainOutput(global_step=5153, training_loss=0.08746906837953448, metrics={'train_runtime': 2920.4938, 'train_samples_per_second': 14.113, 'train_steps_per_second': 1.764, 'total_flos': 6274854343802880.0, 'train_loss': 0.08746906837953448, 'epoch': 1.0})

In [14]:
# Save the model and tokenizer
trainer.save_model("my_trained_codet5")
tokenizer.save_pretrained("my_trained_codet5")

# This will load your trained model and tokenizer, allowing you to use them for inference or further training.
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained("my_trained_codet5")
# tokenizer = AutoTokenizer.from_pretrained("my_trained_codet5")

('my_trained_codet5/tokenizer_config.json',
 'my_trained_codet5/special_tokens_map.json',
 'my_trained_codet5/vocab.json',
 'my_trained_codet5/merges.txt',
 'my_trained_codet5/added_tokens.json',
 'my_trained_codet5/tokenizer.json')